<a href="https://colab.research.google.com/github/thislis/XAI_Sentiment_Analysis/blob/main/test_multi_Bilstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

test

build : Bi-LSTM, Multi classification

In [1]:
#google drive mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Mecab-ko download

!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 9.50 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 4.7 MB/s 
     |████████████████████████████████| 453 kB 80.4 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-10-25 09:25:45--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c5:2ef4, 2406:da00:ff00::22cd:e0db, ...
Connecting to bitbucket.org (bi

In [3]:
#import modules

#test data reading and processing
import re, os
import pandas as pd
import numpy as np
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

#modeling
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import keras.backend as K

In [4]:
#data loading

data_file_path = "/content/drive/Shareddrives/R&E/dataset"
sentiment_file_list = os.listdir(data_file_path)
train_data = pd.DataFrame(columns = ['감정', '문장'])

for path in sentiment_file_list:
  train_add = pd.read_csv(data_file_path + '/' + path)
  train_add.dropna()
  train_data = pd.concat([train_data, train_add], ignore_index = True)

train_data.drop_duplicates(subset=['문장'], inplace=True)#delete overlap data

test_data = pd.read_csv(data_file_path + '/test_dataset.csv', names=['감정', '문장'], skiprows=1)
test_data.drop_duplicates(subset=['문장'], inplace=True) #delete overlap data

In [5]:
#regex, blank to Null

train_data['문장'] = train_data['문장'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data['문장'].replace('', np.nan, inplace=True)
test_data['문장'] = test_data['문장'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_data['문장'].replace('', np.nan, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [6]:
#def stopword

stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '만', '되', '음', '면']

In [7]:
#def function - sentiment to int

def toint(text):
  if text == "분노":
    return 0

  elif text == "불안":
    return 1

  elif text == "당황":
    return 2

  elif text == "기쁨":
    return 3

  elif text == "상처":
    return 4

  elif text == "슬픔":
    return 5

In [8]:
#sentiment to int

train_data['감정'] = train_data['감정'].apply(toint)
test_data['감정'] = test_data['감정'].apply(toint)

print(train_data)
print(test_data)

        감정                                                 문장  Unnamed: 0
0        3                           아내가 드디어 출산하게 되어서 정말 신이 나         0.0
1        3        재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야 너무 행복해         3.0
2        3                        빚을 드디어 다 갚게 되어서 이제야 안도감이 들어         4.0
3        3        남편이 죽기 전에 손자들 얼굴 보고 싶다고 했는데 다행히 늦지 않아서 안도했어        14.0
4        3                                  친구가 사준 로또가 당첨이 됐어        23.0
...     ..                                                ...         ...
114001   5               무기력한 기분을 떨치고 적극적으로 반항하고 주위에 도움을 청해볼래     11833.0
114002   5                  생활이 좀 빠듯하네 이 정도로 적게 받을 줄 몰랐는데 말이야     11842.0
114003   5  아내에게 가서 솔직하게 내 마음을 말해야겠어 다음부터는 뭐든 말해주고 의논했으면 좋겠다고     11857.0
114004   5                          친구들을 만나고 싶어도 만날 수 없어서 속상해     11862.0
114005   5                 친한 친구는 아니지만 젊은 나이에 사망해서 더 마음이 비통하네     11867.0

[108728 rows x 3 columns]
       감정                                                 문장
9       3               

In [9]:
# Train Data Labels
train_data["감정"] = train_data["감정"].astype('category')
train_data["emotion_label"] = train_data["감정"].cat.codes
train_features, train_labels = train_data['문장'], tf.one_hot(train_data["emotion_label"], 6)

# Test Data Labels
test_data["감정"] = test_data["감정"].astype('category')
test_data["emotion_label"] = test_data["감정"].cat.codes
test_features, test_labels = test_data['문장'], tf.one_hot(test_data["emotion_label"], 6)

In [10]:
#tokenizing

mecab = Mecab() 

train_data['tokenized'] = train_data['문장'].apply(mecab.morphs)
train_data['tokenized'] = train_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
test_data['tokenized'] = test_data['문장'].apply(mecab.morphs)
test_data['tokenized'] = test_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

In [11]:
#tokenized seq-> X

X_train = train_data['tokenized'].values
X_test= test_data['tokenized'].values

In [12]:
#max length for LSTM

max_len = 35

In [13]:
#F1-score

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [14]:
#model fitting

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 2
total_cnt = len(tokenizer.word_index) #word count
rare_cnt = 0 #count word; frequency of appearance < threshold
total_freq = 0 #sum freq in all train data
rare_freq = 0 #sum freq in train data; frequency of appearance < threshold

#(word, frequency) ->key and value
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

vocab_size = total_cnt - rare_cnt + 2

tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Bidirectional(LSTM(hidden_units))) # Bidirectional LSTM
model.add(Dense(6, activation='softmax'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop',loss='categorical_crossentropy',
              metrics=['acc',tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), get_f1])
history = model.fit(X_train, train_labels, epochs=15, callbacks=[es, mc], batch_size=128, validation_split=0.2)

Epoch 1/15
676/680 [============================>.] - ETA: 0s - loss: 1.3371 - acc: 0.4782 - recall: 0.2469 - precision: 0.7743 - get_f1: 0.3584
Epoch 1: val_acc improved from -inf to 0.32484, saving model to best_model.h5
680/680 [==============================] - 20s 13ms/step - loss: 1.3364 - acc: 0.4785 - recall: 0.2475 - precision: 0.7744 - get_f1: 0.3592 - val_loss: 2.1877 - val_acc: 0.3248 - val_recall: 0.2131 - val_precision: 0.6460 - val_get_f1: 0.2943
Epoch 2/15
676/680 [============================>.] - ETA: 0s - loss: 1.1670 - acc: 0.5535 - recall: 0.3653 - precision: 0.7724 - get_f1: 0.4951
Epoch 2: val_acc improved from 0.32484 to 0.34195, saving model to best_model.h5
680/680 [==============================] - 8s 12ms/step - loss: 1.1671 - acc: 0.5535 - recall: 0.3653 - precision: 0.7724 - get_f1: 0.4950 - val_loss: 2.1256 - val_acc: 0.3419 - val_recall: 0.2284 - val_precision: 0.6591 - val_get_f1: 0.3164
Epoch 3/15
680/680 [==============================] - ETA: 0s - lo

In [15]:
loaded_model = load_model('best_model.h5', custom_objects={'get_f1' : get_f1})
loaded_model.evaluate(X_test, test_labels)

373/373 [==============================] - 3s 5ms/step - loss: 1.4216 - acc: 0.5075 - recall: 0.3693 - precision: 0.7081 - get_f1: 0.4622


[1.421587347984314,
 0.5074589252471924,
 0.36934295296669006,
 0.7080655694007874,
 0.4621593952178955]